In [1]:
import numpy as np
import pandas as pd
import os
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
drivers = pd.read_csv('data/drivers.csv')
drivers['driver'] = drivers['forename'] + ' ' + drivers['surname']
drivers = drivers[['driverId', 'driver']]

In [3]:
constructors = pd.read_csv('data/constructors.csv')
constructors = constructors.rename(columns={'name': 'constructor'})
constructors = constructors[['constructorId', 'constructor']]

In [4]:
circuits = pd.read_csv('data/circuits.csv')
circuits = circuits[['circuitId', 'name', 'country']]
circuits = circuits.rename(columns={'name': 'circuit', 'country': 'circuit_country'})

In [5]:
races = pd.read_csv('data/races.csv')
races = races.merge(circuits, how='left', on='circuitId')
races = races[['raceId', 'year', 'round', 'date', 'circuit', 'circuit_country']]
races['date'] = pd.to_datetime(races['date'], format='%Y-%m-%d').map(lambda x: x.date())

In [6]:
results = pd.read_csv('data/results.csv')
results.sort_values('raceId').head()
results = results.merge(drivers, how='left', on='driverId')
results = results.merge(constructors, how='left', on='constructorId')
results = results.merge(races, how='left', on='raceId')
results = results[['resultId', 'year', 'round', 'date', 'circuit', 'circuit_country', 'raceId',
                   'positionOrder', 'position', 'driver', 'constructor', 'points', 'grid']]
results = results.sort_values(['year', 'round', 'positionOrder'])

In [7]:
finished_results = results.loc[results['position'] != '\\N']
finished_results['position'] = finished_results['position'].astype(int)

In [8]:
teams_1 = finished_results[['resultId', 'raceId', 'driver', 'position']]
teams_1 = teams_1.rename(columns={'resultId': 'resultId1', 'driver': 'driver1', 'position': 'position1'})

teams_2 = finished_results[['resultId', 'raceId', 'driver', 'position']]
teams_2 = teams_2.rename(columns={'resultId': 'resultId2', 'driver': 'driver2', 'position': 'position2'})

pvp_results = teams_1.merge(teams_2, how='inner', on='raceId')
pvp_results = pvp_results.loc[pvp_results['resultId1'] < pvp_results['resultId2']]
pvp_results = pvp_results.loc[pvp_results['position1'] != pvp_results['position2']]
pvp_results = pvp_results.loc[pvp_results['driver1'] != pvp_results['driver2']]
pvp_results['score'] = pvp_results['position2'].astype(str) + ' : ' + pvp_results['position1'].astype(str)
pvp_results = pvp_results.drop(columns=['resultId1', 'resultId2'])

In [9]:
race_stats = finished_results.groupby(['raceId', 'year', 'round', 'date']).agg(num_drivers=('driver', 'nunique')).reset_index()
pvp_results = pvp_results.merge(race_stats, how='left', on='raceId')

year_stats = finished_results.groupby(['year']).agg(num_races=('raceId', 'nunique')).reset_index()
pvp_results = pvp_results.merge(year_stats, how='left', on='year')

In [10]:
pvp_results.to_csv('data/pvp_results.csv', index=False)

In [11]:
pvp_results = pvp_results.rename(columns={
    'driver1': 'home_team',
    'driver2': 'away_team',
    'date': 'game_date',
    'year': 'season',
})
pvp_results = pvp_results.drop(columns=['position1', 'position2', 'raceId'])
pvp_results['home_field'] = 'no'
pvp_results['country'] = 'world'
pvp_results['tournament'] = 'F1'

In [12]:
pvp_results.to_csv('data/listing.csv', index=False, sep=';')

In [13]:
pvp_results

,home_team,away_team,score,season,round,game_date,num_drivers,num_races,home_field,country,tournament
0,Nino Farina,Luigi Fagioli,2 : 1,1950,1,1950-05-13,12,7,no,world,F1
1,Nino Farina,Reg Parnell,3 : 1,1950,1,1950-05-13,12,7,no,world,F1
2,Nino Farina,Yves Cabantous,4 : 1,1950,1,1950-05-13,12,7,no,world,F1
3,Nino Farina,Louis Rosier,5 : 1,1950,1,1950-05-13,12,7,no,world,F1
4,Nino Farina,Bob Gerard,6 : 1,1950,1,1950-05-13,12,7,no,world,F1
...,...,...,...,...,...,...,...,...,...,...,...
111498,Lance Stroll,Kevin Magnussen,16 : 14,2024,24,2024-12-08,17,24,no,world,F1
111499,Lance Stroll,Liam Lawson,17 : 14,2024,24,2024-12-08,17,24,no,world,F1
111500,Jack Doohan,Kevin Magnussen,16 : 15,2024,24,2024-12-08,17,24,no,world,F1
111501,Jack Doohan,Liam Lawson,17 : 15,2024,24,2024-12-08,17,24,no,world,F1


In [14]:
# season_info = pvp_results.groupby(['season']).agg(
#     min_date=('game_date', 'min'),
#     max_date=('game_date', 'max'),
# ).reset_index()
# season_info['first_date'] = season_info['season'].map(lambda x: dt.date(x, 1, 1))
# season_info['last_date'] = season_info['season'].map(lambda x: dt.date(x, 12, 31))
# season_info['days_from_start'] = (season_info['min_date'] - season_info['first_date']).map(lambda x: x.days)
# season_info['days_to_end'] = (season_info['last_date'] - season_info['max_date']).map(lambda x: x.days)

In [15]:
from pandas import (
    DataFrame,
    read_csv,
    to_datetime,
)

In [16]:
home_drivers = pvp_results[['home_team', 'season', 'round']].rename(columns={'home_team': 'driver'})
home_drivers['race'] = home_drivers['season'].astype(str) + '_' + home_drivers['round'].astype(str)

away_drivers = pvp_results[['away_team', 'season', 'round']].rename(columns={'away_team': 'driver'})
away_drivers['race'] = away_drivers['season'].astype(str) + '_' + away_drivers['round'].astype(str)

all_drivers = pd.concat([home_drivers, away_drivers], ignore_index=True)

In [17]:
drivers_stats = all_drivers.groupby(['driver']).agg(
    num_races=('race', 'nunique'),
    num_pvps=('race', 'count'),
).reset_index()
drivers_stats = drivers_stats.sort_values(['num_pvps', 'num_races'], ascending=[False, False])

In [18]:
drivers_stats.tail(160)

,driver,num_races,num_pvps
422,Mauri Rose,1,24
447,Myron Fohr,1,24
627,Travis Webb,1,24
639,Walt Ader,1,24
640,Walt Brown,1,24
...,...,...,...
658,Óscar González,1,7
143,Dorino Serafini,1,6
243,Harry Blanchard,1,6
441,Mike MacDowel,1,6
